In [7]:
import pandas as pd
import networkx as nx

In [8]:
G = nx.Graph()

## Datasets

- brasileirao_serie_a.csv
- brazilian-soccer-database
- campeonato-paulista-de-futebol-placares-e-odds
- copa_do_brasil.csv
- englishfootball
- LaLigaCompleteDataset
- Ligue1_saison_2020_2021
- worldcup
- tennis
- basketball

In [9]:
# https://basedosdados.org/dataset/c861330e-bca2-474d-9073-bc70744a1b23?table=18835b0d-233e-4857-b454-1fa34a81b4fa

brasileirao = pd.read_csv('../data/brasileirao_a/brasileirao_serie_a.csv', sep=',')
brasileirao.columns

Index(['ano_campeonato', 'data', 'rodada', 'estadio', 'arbitro', 'publico',
       'publico_max', 'time_mandante', 'time_visitante', 'tecnico_mandante',
       'tecnico_visitante', 'colocacao_mandante', 'colocacao_visitante',
       'valor_equipe_titular_mandante', 'valor_equipe_titular_visitante',
       'idade_media_titular_mandante', 'idade_media_titular_visitante',
       'gols_mandante', 'gols_visitante', 'gols_1_tempo_mandante',
       'gols_1_tempo_visitante', 'escanteios_mandante', 'escanteios_visitante',
       'faltas_mandante', 'faltas_visitante', 'chutes_bola_parada_mandante',
       'chutes_bola_parada_visitante', 'defesas_mandante', 'defesas_visitante',
       'impedimentos_mandante', 'impedimentos_visitante', 'chutes_mandante',
       'chutes_visitante', 'chutes_fora_mandante', 'chutes_fora_visitante'],
      dtype='object')

In [31]:
#df = brasileirao[brasileirao['ano_campeonato'] == 2017]
df = brasileirao.copy()
df = df[['ano_campeonato', 'time_mandante', 'time_visitante', 'gols_mandante', 'gols_visitante']] 
df.head()

,ano_campeonato,time_mandante,time_visitante,gols_mandante,gols_visitante
0,2017,Chapecoense,Palmeiras,1.0,0.0
1,2017,Chapecoense,Avaí FC,2.0,0.0
2,2017,Chapecoense,Grêmio,3.0,6.0
3,2017,Chapecoense,Vasco da Gama,2.0,1.0
4,2017,Chapecoense,Botafogo,0.0,2.0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8079 entries, 0 to 8078
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ano_campeonato  8079 non-null   int64  
 1   time_mandante   8079 non-null   object 
 2   time_visitante  8079 non-null   object 
 3   gols_mandante   8078 non-null   float64
 4   gols_visitante  8078 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 315.7+ KB


In [12]:
vertices = list(df['time_mandante'].unique())
print(vertices)
print(len(vertices))

['Chapecoense', 'Palmeiras', 'EC Bahia', 'EC Vitória', 'Atlético-PR', 'Grêmio', 'Corinthians', 'Sport Recife', 'Atlético-GO', 'Vasco da Gama', 'Flamengo', 'Santos FC', 'Fluminense', 'Atlético-MG', 'Ponte Preta', 'Avaí FC', 'Coritiba FC', 'Botafogo', 'São Paulo', 'Cruzeiro', 'Paraná', 'Fortaleza', 'Goiás EC', 'Juventude', 'Paysandu SC', 'São Caetano', 'Internacional', 'Brasiliense-DF', 'Figueirense FC', 'Santa Cruz', 'América-MG', 'Náutico', 'América-RN', 'Portuguesa', 'Ipatinga FC', 'Ceará SC', 'Athletico-PR', 'RB Bragantino', 'Santo André', 'Barueri', 'CSA', 'Santos', 'Cuiabá-MT', 'Guarani', 'Criciúma EC', 'Joinville-SC', 'Goiás']
47


In [13]:
df['pontos_mandante'] = 0.0
df['pontos_visitante'] = 0.0

In [32]:
df.loc[df['gols_mandante'] > df['gols_visitante'], 'pontos_mandante'] = 3.0
df.loc[df['gols_mandante'] < df['gols_visitante'], 'pontos_visitante'] = 3.0
df.loc[df['gols_mandante'] == df['gols_visitante'], 'pontos_mandante'] = 1.0
df.loc[df['gols_mandante'] == df['gols_visitante'], 'pontos_visitante'] = 1.0
df.head()

,ano_campeonato,time_mandante,time_visitante,gols_mandante,gols_visitante,pontos_mandante,pontos_visitante
0,2017,Chapecoense,Palmeiras,1.0,0.0,3.0,NaN
1,2017,Chapecoense,Avaí FC,2.0,0.0,3.0,NaN
2,2017,Chapecoense,Grêmio,3.0,6.0,NaN,3.0
3,2017,Chapecoense,Vasco da Gama,2.0,1.0,3.0,NaN
4,2017,Chapecoense,Botafogo,0.0,2.0,NaN,3.0


In [15]:
df['pesos'] = 0.0

In [16]:
df.loc[df['gols_mandante'] > df['gols_visitante'], 'pesos'] = 3.0
df.loc[df['gols_mandante'] == df['gols_visitante'], 'pesos'] = 1.0
df

,ano_campeonato,time_mandante,time_visitante,gols_mandante,gols_visitante,pontos_mandante,pontos_visitante,pesos
0,2017,Chapecoense,Palmeiras,1.0,0.0,3.0,0.0,3.0
1,2017,Chapecoense,Avaí FC,2.0,0.0,3.0,0.0,3.0
2,2017,Chapecoense,Grêmio,3.0,6.0,0.0,3.0,0.0
3,2017,Chapecoense,Vasco da Gama,2.0,1.0,3.0,0.0,3.0
4,2017,Chapecoense,Botafogo,0.0,2.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...
8074,2011,Flamengo,Atlético-GO,1.0,4.0,0.0,3.0,0.0
8075,2011,Santos FC,Flamengo,4.0,5.0,0.0,3.0,0.0
8076,2011,América-MG,Internacional,2.0,4.0,0.0,3.0,0.0
8077,2011,Coritiba FC,São Paulo,3.0,4.0,0.0,3.0,0.0


In [17]:
for t in vertices:
    man = df.loc[df['time_mandante'] == t, "pontos_mandante"].sum()
    vis = df.loc[df['time_visitante'] == t, "pontos_visitante"].sum()
    pontos = man + vis
    print(t, pontos)

Chapecoense 287.0
Palmeiras 1137.0
EC Bahia 485.0
EC Vitória 458.0
Atlético-PR 797.0
Grêmio 1095.0
Corinthians 1166.0
Sport Recife 500.0
Atlético-GO 295.0
Vasco da Gama 773.0
Flamengo 1231.0
Santos FC 1056.0
Fluminense 1146.0
Atlético-MG 1129.0
Ponte Preta 416.0
Avaí FC 271.0
Coritiba FC 681.0
Botafogo 909.0
São Paulo 1273.0
Cruzeiro 1037.0
Paraná 275.0
Fortaleza 376.0
Goiás EC 652.0
Juventude 306.0
Paysandu SC 134.0
São Caetano 204.0
Internacional 1165.0
Brasiliense-DF 39.0
Figueirense FC 513.0
Santa Cruz 59.0
América-MG 230.0
Náutico 200.0
América-RN 17.0
Portuguesa 131.0
Ipatinga FC 35.0
Ceará SC 308.0
Athletico-PR 270.0
RB Bragantino 205.0
Santo André 41.0
Barueri 80.0
CSA 32.0
Santos 130.0
Cuiabá-MT 125.0
Guarani 129.0
Criciúma EC 177.0
Joinville-SC 31.0
Goiás 77.0


In [93]:
# https://www.kaggle.com/datasets/ricardomattos05/brazilian-soccer-database

libertadores = pd.read_csv('../data/BrazilianSoccer/Data/Libertadores_Matches.csv', sep=',')
libertadores.head()

,datetime,home_team,away_team,home_goal,away_goal,season,stage
0,2013-02-12 20:15:00,Nacional (URU),Barcelona-EQU,2,2,2013.0,group stage
1,2013-02-13 22:00:00,Boca Juniors,Toluca,1,2,2013.0,group stage
2,2013-02-19 23:45:00,Toluca,Nacional (URU),2,3,2013.0,group stage
3,2013-02-27 19:45:00,Barcelona-EQU,Boca Juniors,1,2,2013.0,group stage
4,2013-03-07 00:15:00,Toluca,Barcelona-EQU,1,1,2013.0,group stage


In [94]:
libertadores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   1254 non-null   object 
 1   home_team  1255 non-null   object 
 2   away_team  1255 non-null   object 
 3   home_goal  1255 non-null   object 
 4   away_goal  1255 non-null   object 
 5   season     1254 non-null   float64
 6   stage      1255 non-null   object 
dtypes: float64(1), object(6)
memory usage: 68.8+ KB


In [109]:
libertadores['home_goal'] = pd.to_numeric(libertadores['home_goal'], errors='coerce')
libertadores['away_goal'] = pd.to_numeric(libertadores['away_goal'], errors='coerce')
libertadores['season'] = pd.to_numeric(libertadores['season'], errors='coerce')
libertadores['stage'] = pd.to_numeric(libertadores['stage'], errors='coerce')

In [110]:
libertadores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   1254 non-null   object 
 1   home_team  1255 non-null   object 
 2   away_team  1255 non-null   object 
 3   home_goal  1253 non-null   float64
 4   away_goal  1253 non-null   float64
 5   season     1254 non-null   float64
 6   stage      0 non-null      float64
dtypes: float64(4), object(3)
memory usage: 68.8+ KB


In [104]:
df = libertadores.copy()
#df = df[df['season'] == 2017]
df = df[['season', 'home_team', 'away_team', 'home_goal', 'away_goal','stage']] 
df.head()

,season,home_team,away_team,home_goal,away_goal,stage
0,2013.0,Nacional (URU),Barcelona-EQU,2.0,2.0,group stage
1,2013.0,Boca Juniors,Toluca,1.0,2.0,group stage
2,2013.0,Toluca,Nacional (URU),2.0,3.0,group stage
3,2013.0,Barcelona-EQU,Boca Juniors,1.0,2.0,group stage
4,2013.0,Toluca,Barcelona-EQU,1.0,1.0,group stage


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   season     1254 non-null   float64
 1   home_team  1255 non-null   object 
 2   away_team  1255 non-null   object 
 3   home_goal  1253 non-null   float64
 4   away_goal  1253 non-null   float64
 5   stage      1255 non-null   object 
dtypes: float64(3), object(3)
memory usage: 59.0+ KB


In [106]:
df['home_points'] = 0.0
df['away_points'] = 0.0

In [107]:
df.loc[df['home_goal'] > df['away_goal'], 'home_points'] = 3.0
df.loc[df['home_goal'] < df['away_goal'], 'away_points'] = 3.0
df.loc[df['home_goal'] == df['away_goal'], 'home_points'] = 1.0
df.loc[df['home_goal'] == df['away_goal'], 'away_points'] = 1.0
df.head()

,season,home_team,away_team,home_goal,away_goal,stage,home_points,away_points
0,2013.0,Nacional (URU),Barcelona-EQU,2.0,2.0,group stage,1.0,1.0
1,2013.0,Boca Juniors,Toluca,1.0,2.0,group stage,0.0,3.0
2,2013.0,Toluca,Nacional (URU),2.0,3.0,group stage,0.0,3.0
3,2013.0,Barcelona-EQU,Boca Juniors,1.0,2.0,group stage,0.0,3.0
4,2013.0,Toluca,Barcelona-EQU,1.0,1.0,group stage,1.0,1.0


In [42]:
df['weights'] = 0.0

In [43]:
df.loc[df['home_goal'] > df['away_goal'], 'weights'] = 3.0
df.loc[df['home_goal'] == df['away_goal'], 'weights'] = 1.0
df

,season,home_team,away_team,home_goal,away_goal,stage,home_points,away_points,weights
0,2013.0,Nacional (URU),Barcelona-EQU,2,2,group stage,1.0,1.0,1.0
1,2013.0,Boca Juniors,Toluca,1,2,group stage,0.0,3.0,0.0
2,2013.0,Toluca,Nacional (URU),2,3,group stage,0.0,3.0,0.0
3,2013.0,Barcelona-EQU,Boca Juniors,1,2,group stage,0.0,3.0,0.0
4,2013.0,Toluca,Barcelona-EQU,1,1,group stage,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
1250,2022.0,Athletico,Palmeiras,1,0,semifinals,3.0,0.0,3.0
1251,2022.0,Palmeiras,Athletico,2,2,semifinals,1.0,1.0,1.0
1252,2022.0,Vélez Sarsfield,Flamengo,0,4,semifinals,0.0,3.0,0.0
1253,2022.0,Flamengo,Vélez Sarsfield,2,1,semifinals,3.0,0.0,3.0


In [45]:
for t in vertices:
    man = df.loc[df['home_team'] == t, "home_points"].sum()
    vis = df.loc[df['away_team'] == t, "away_points"].sum()
    pontos = man + vis
    print(t, pontos)

Chapecoense 7.0
Palmeiras 168.0
EC Bahia 0.0
EC Vitória 0.0
Atlético-PR 19.0
Grêmio 135.0
Corinthians 66.0
Sport Recife 0.0
Atlético-GO 0.0
Vasco da Gama 0.0
Flamengo 135.0
Santos FC 0.0
Fluminense 34.0
Atlético-MG 135.0
Ponte Preta 0.0
Avaí FC 0.0
Coritiba FC 0.0
Botafogo 24.0
São Paulo 60.0
Cruzeiro 64.0
Paraná 0.0
Fortaleza 11.0
Goiás EC 0.0
Juventude 0.0
Paysandu SC 0.0
São Caetano 0.0
Internacional 67.0
Brasiliense-DF 0.0
Figueirense FC 0.0
Santa Cruz 0.0
América-MG 2.0
Náutico 0.0
América-RN 0.0
Portuguesa 0.0
Ipatinga FC 0.0
Ceará SC 0.0
Athletico-PR 0.0
RB Bragantino 0.0
Santo André 0.0
Barueri 0.0
CSA 0.0
Santos 63.0
Cuiabá-MT 0.0
Guarani 0.0
Criciúma EC 0.0
Joinville-SC 0.0
Goiás 0.0


In [46]:
# https://www.kaggle.com/datasets/profdavidsonsantana/campeonato-paulista-de-futebol-placares-e-odds

paulista = pd.read_csv('../data/campeonato_paulista/paulista-2015.csv', sep=';')
df = paulista.copy()
df.head() 

,nacao,campeonato,rodada,ano,dia,home,away,HGHT,AGHT,HGFT,AGFT,odd_home,odd_draw,odd_away,over25,endereco
0,BRAZIL,PAULISTA,ROUND 1,2015,31.01.2015,Rio Claro,Botafogo SP,0,0,0,1,2.72,3.23,2.57,1.97,https://www.flashscore.com/match/n9Mm42Er/#/ma...
1,BRAZIL,PAULISTA,ROUND 1,2015,31.01.2015,Bragantino,Sao Bernardo,0,0,1,0,2.16,3.37,3.33,1.93,https://www.flashscore.com/match/lYtDA4yF/#/ma...
2,BRAZIL,PAULISTA,ROUND 1,2015,31.01.2015,Audax-SP,Palmeiras,0,3,1,3,5.32,3.85,1.62,1.83,https://www.flashscore.com/match/2yoH9OjL/#/ma...
3,BRAZIL,PAULISTA,ROUND 1,2015,31.01.2015,Capivariano,RB Brasil,0,1,0,1,2.37,3.27,2.97,2.08,https://www.flashscore.com/match/fBs9BpM8/#/ma...
4,BRAZIL,PAULISTA,ROUND 1,2015,31.01.2015,Piracicaba,Mogi Mirim,0,1,0,1,2.50,3.23,2.85,1.87,https://www.flashscore.com/match/GKr5CQ62/#/ma...


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   nacao       158 non-null    object 
 1   campeonato  158 non-null    object 
 2   rodada      158 non-null    object 
 3   ano         158 non-null    int64  
 4   dia         158 non-null    object 
 5   home        158 non-null    object 
 6   away        158 non-null    object 
 7   HGHT        158 non-null    int64  
 8   AGHT        158 non-null    int64  
 9   HGFT        158 non-null    int64  
 10  AGFT        158 non-null    int64  
 11  odd_home    158 non-null    float64
 12  odd_draw    158 non-null    float64
 13  odd_away    158 non-null    float64
 14  over25      158 non-null    float64
 15  endereco    158 non-null    object 
dtypes: float64(4), int64(5), object(7)
memory usage: 19.9+ KB


In [47]:
df = df[['ano','home','away','HGHT','AGHT','HGFT','AGFT']]
df.head()

,ano,home,away,HGHT,AGHT,HGFT,AGFT
0,2015,Rio Claro,Botafogo SP,0,0,0,1
1,2015,Bragantino,Sao Bernardo,0,0,1,0
2,2015,Audax-SP,Palmeiras,0,3,1,3
3,2015,Capivariano,RB Brasil,0,1,0,1
4,2015,Piracicaba,Mogi Mirim,0,1,0,1


In [48]:
vertices = list(df['home'].unique())
print(vertices)
print(len(vertices))

['Rio Claro', 'Bragantino', 'Audax-SP', 'Capivariano', 'Piracicaba', 'Penapolense', 'Corinthians', 'Sao Bento', 'Santos', 'Ponte Preta', 'Sao Bernardo', 'Botafogo SP', 'Portuguesa', 'Ituano', 'Sao Paulo', 'RB Brasil', 'Marilia', 'Mogi Mirim', 'Palmeiras', 'Linense']
20


In [49]:
df['home_points'] = 0.0
df['away_points'] = 0.0

In [50]:
df.loc[df['HGHT'] > df['AGHT'], 'home_points'] = 3.0
df.loc[df['HGHT'] < df['AGHT'], 'away_points'] = 3.0
df.loc[df['HGHT'] == df['AGHT'], 'home_points'] = 1.0
df.loc[df['HGHT'] == df['AGHT'], 'away_points'] = 1.0
df.head()

,ano,home,away,HGHT,AGHT,HGFT,AGFT,home_points,away_points
0,2015,Rio Claro,Botafogo SP,0,0,0,1,1.0,1.0
1,2015,Bragantino,Sao Bernardo,0,0,1,0,1.0,1.0
2,2015,Audax-SP,Palmeiras,0,3,1,3,0.0,3.0
3,2015,Capivariano,RB Brasil,0,1,0,1,0.0,3.0
4,2015,Piracicaba,Mogi Mirim,0,1,0,1,0.0,3.0


In [51]:
for t in vertices:
    man = df.loc[df['home'] == t, "home_points"].sum()
    vis = df.loc[df['away'] == t, "away_points"].sum()
    pontos = man + vis
    print(t, pontos)

Rio Claro 14.0
Bragantino 11.0
Audax-SP 17.0
Capivariano 15.0
Piracicaba 14.0
Penapolense 18.0
Corinthians 43.0
Sao Bento 21.0
Santos 38.0
Ponte Preta 26.0
Sao Bernardo 21.0
Botafogo SP 18.0
Portuguesa 17.0
Ituano 19.0
Sao Paulo 29.0
RB Brasil 26.0
Marilia 5.0
Mogi Mirim 20.0
Palmeiras 29.0
Linense 10.0


In [52]:
df['weights'] = 0

In [53]:
df.loc[df['HGHT'] > df['AGHT'], 'weights'] = 3
df.loc[df['HGHT'] == df['AGHT'], 'weights'] = 1
df.head()

,ano,home,away,HGHT,AGHT,HGFT,AGFT,home_points,away_points,weights
0,2015,Rio Claro,Botafogo SP,0,0,0,1,1.0,1.0,1
1,2015,Bragantino,Sao Bernardo,0,0,1,0,1.0,1.0,1
2,2015,Audax-SP,Palmeiras,0,3,1,3,0.0,3.0,0
3,2015,Capivariano,RB Brasil,0,1,0,1,0.0,3.0,0
4,2015,Piracicaba,Mogi Mirim,0,1,0,1,0.0,3.0,0


In [54]:
# https://basedosdados.org/dataset/c861330e-bca2-474d-9073-bc70744a1b23?table=19e96c3f-c932-4e14-82e2-58e170454e3d 

copa_brasil = pd.read_csv('../data/copa_brasil/copa_brasil.csv', sep=',')
copa_brasil.head()

,ano_campeonato,data,horario,fase,tipo_fase,estadio,arbitro,publico,publico_max,time_mandante,...,chutes_bola_parada_mandante,chutes_bola_parada_visitante,defesas_mandante,defesas_visitante,impedimentos_mandante,impedimentos_visitante,chutes_mandante,chutes_visitante,chutes_fora_mandante,chutes_fora_visitante
0,2020,2020-12-23,20:30:00,Semi-Finais,Jogos De Ida,Arena do Grêmio,NaN,0,60540.0,Grêmio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,2020-10-28,18:00:00,Oitavas De Final,Jogos De Ida,Estádio Antonio Accioly,NaN,0,13000.0,Atlético-GO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,2020-10-28,20:30:00,Oitavas De Final,Jogos De Ida,Arena do Grêmio,NaN,0,60540.0,Grêmio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,2020-09-23,21:30:00,4ª Eliminatória,Jogos De Volta,Estádio São Januário,NaN,0,24584.0,Vasco da Gama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,2020-08-25,16:00:00,3ª Eliminatória,Jogos De Volta,Estádio Vianão,NaN,0,1735.0,Afogados,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df = copa_brasil.copy()
df.head()

,ano_campeonato,data,horario,fase,tipo_fase,estadio,arbitro,publico,publico_max,time_mandante,...,chutes_bola_parada_mandante,chutes_bola_parada_visitante,defesas_mandante,defesas_visitante,impedimentos_mandante,impedimentos_visitante,chutes_mandante,chutes_visitante,chutes_fora_mandante,chutes_fora_visitante
0,2020,2020-12-23,20:30:00,Semi-Finais,Jogos De Ida,Arena do Grêmio,NaN,0,60540.0,Grêmio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,2020-10-28,18:00:00,Oitavas De Final,Jogos De Ida,Estádio Antonio Accioly,NaN,0,13000.0,Atlético-GO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,2020-10-28,20:30:00,Oitavas De Final,Jogos De Ida,Arena do Grêmio,NaN,0,60540.0,Grêmio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,2020-09-23,21:30:00,4ª Eliminatória,Jogos De Volta,Estádio São Januário,NaN,0,24584.0,Vasco da Gama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,2020-08-25,16:00:00,3ª Eliminatória,Jogos De Volta,Estádio Vianão,NaN,0,1735.0,Afogados,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
#df = df[df['ano_campeonato'] == 2017]
df = df[['ano_campeonato', 'time_mandante', 'time_visitante', 'gols_mandante', 'gols_visitante']] 
df.head()

,ano_campeonato,time_mandante,time_visitante,gols_mandante,gols_visitante
0,2020,Grêmio,São Paulo,1,0
1,2020,Atlético-GO,Internacional,1,2
2,2020,Grêmio,Juventude,1,0
3,2020,Vasco da Gama,Botafogo,0,0
4,2020,Afogados,Ponte Preta,0,2


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ano_campeonato  424 non-null    int64 
 1   time_mandante   424 non-null    object
 2   time_visitante  424 non-null    object
 3   gols_mandante   424 non-null    int64 
 4   gols_visitante  424 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 16.7+ KB


In [59]:
df['pontos_mandante'] = 0.0
df['pontos_visitante'] = 0.0

In [60]:
df.loc[df['gols_mandante'] > df['gols_visitante'], 'pontos_mandante'] = 3
df.loc[df['gols_mandante'] < df['gols_visitante'], 'pontos_visitante'] = 3
df.loc[df['gols_mandante'] == df['gols_visitante'], 'pontos_mandante'] = 1
df.loc[df['gols_mandante'] == df['gols_visitante'], 'pontos_visitante'] = 1
df.head()

,ano_campeonato,time_mandante,time_visitante,gols_mandante,gols_visitante,pontos_mandante,pontos_visitante
0,2020,Grêmio,São Paulo,1,0,3.0,0.0
1,2020,Atlético-GO,Internacional,1,2,0.0,3.0
2,2020,Grêmio,Juventude,1,0,3.0,0.0
3,2020,Vasco da Gama,Botafogo,0,0,1.0,1.0
4,2020,Afogados,Ponte Preta,0,2,0.0,3.0


In [61]:
df['pesos'] = 0.0

In [62]:
df.loc[df['gols_mandante'] > df['gols_visitante'], 'pesos'] = 3.0
df.loc[df['gols_mandante'] == df['gols_visitante'], 'pesos'] = 1.0
df.head()

,ano_campeonato,time_mandante,time_visitante,gols_mandante,gols_visitante,pontos_mandante,pontos_visitante,pesos
0,2020,Grêmio,São Paulo,1,0,3.0,0.0,3.0
1,2020,Atlético-GO,Internacional,1,2,0.0,3.0,0.0
2,2020,Grêmio,Juventude,1,0,3.0,0.0,3.0
3,2020,Vasco da Gama,Botafogo,0,0,1.0,1.0,1.0
4,2020,Afogados,Ponte Preta,0,2,0.0,3.0,0.0


In [63]:
# https://github.com/jfjelstul/englishfootball

englishfootball = pd.read_csv('../data/englishfootball/data-csv/matches.csv', sep=',')
englishfootball.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203956 entries, 0 to 203955
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   key_id                  203956 non-null  int64 
 1   season_id               203956 non-null  object
 2   season                  203956 non-null  int64 
 3   tier                    203956 non-null  int64 
 4   division                203956 non-null  object
 5   subdivision             203956 non-null  object
 6   match_id                203956 non-null  object
 7   match_name              203956 non-null  object
 8   home_team_id            203956 non-null  object
 9   home_team_name          203956 non-null  object
 10  away_team_id            203956 non-null  object
 11  away_team_name          203956 non-null  object
 12  score                   203956 non-null  object
 13  home_team_score         203956 non-null  int64 
 14  away_team_score         203956 non-n

In [64]:
df = englishfootball.copy()
df.head()

,key_id,season_id,season,tier,division,subdivision,match_id,match_name,home_team_id,home_team_name,...,away_team_name,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,result,home_team_win,away_team_win,draw
0,1,S-1888-1,1888,1,First Division,None,M-1888-1-001,Accrington vs Aston Villa,T-001,Accrington,...,Aston Villa,1-1,1,1,0,0,draw,0,0,1
1,2,S-1888-1,1888,1,First Division,None,M-1888-1-002,Accrington vs Blackburn Rovers,T-001,Accrington,...,Blackburn Rovers,0-2,0,2,-2,2,away team win,0,1,0
2,3,S-1888-1,1888,1,First Division,None,M-1888-1-003,Accrington vs Bolton Wanderers,T-001,Accrington,...,Bolton Wanderers,2-3,2,3,-1,1,away team win,0,1,0
3,4,S-1888-1,1888,1,First Division,None,M-1888-1-004,Accrington vs Burnley,T-001,Accrington,...,Burnley,5-1,5,1,4,-4,home team win,1,0,0
4,5,S-1888-1,1888,1,First Division,None,M-1888-1-005,Accrington vs Derby County,T-001,Accrington,...,Derby County,6-2,6,2,4,-4,home team win,1,0,0


In [65]:
#df = df[df['season'] == 2017]
df = df[['season', 'home_team_name', 'away_team_name', 'home_team_score', 'away_team_score']] 
df.head()

,season,home_team_name,away_team_name,home_team_score,away_team_score
0,1888,Accrington,Aston Villa,1,1
1,1888,Accrington,Blackburn Rovers,0,2
2,1888,Accrington,Bolton Wanderers,2,3
3,1888,Accrington,Burnley,5,1
4,1888,Accrington,Derby County,6,2


In [66]:
df['home_points'] = 0.0
df['away_points'] = 0.0

In [67]:
df.loc[df['home_team_score'] > df['away_team_score'], 'home_points'] = 3.0
df.loc[df['home_team_score'] < df['away_team_score'], 'away_points'] = 3.0
df.loc[df['home_team_score'] == df['away_team_score'], 'home_points'] = 1.0
df.loc[df['home_team_score'] == df['away_team_score'], 'away_points'] = 1.0
df.head()

,season,home_team_name,away_team_name,home_team_score,away_team_score,home_points,away_points
0,1888,Accrington,Aston Villa,1,1,1.0,1.0
1,1888,Accrington,Blackburn Rovers,0,2,0.0,3.0
2,1888,Accrington,Bolton Wanderers,2,3,0.0,3.0
3,1888,Accrington,Burnley,5,1,3.0,0.0
4,1888,Accrington,Derby County,6,2,3.0,0.0


In [68]:
df['weights'] = 0.0

In [69]:
df.loc[df['home_team_score'] > df['away_team_score'], 'weights'] = 3.0
df.loc[df['home_team_score'] == df['away_team_score'], 'weights'] = 1.0
df.head()

,season,home_team_name,away_team_name,home_team_score,away_team_score,home_points,away_points,weights
0,1888,Accrington,Aston Villa,1,1,1.0,1.0,1.0
1,1888,Accrington,Blackburn Rovers,0,2,0.0,3.0,0.0
2,1888,Accrington,Bolton Wanderers,2,3,0.0,3.0,0.0
3,1888,Accrington,Burnley,5,1,3.0,0.0,3.0
4,1888,Accrington,Derby County,6,2,3.0,0.0,3.0


In [70]:
vertices = list(df['home_team_name'].unique())
print(vertices)
print(len(vertices))

['Accrington', 'Aston Villa', 'Blackburn Rovers', 'Bolton Wanderers', 'Burnley', 'Derby County', 'Everton', 'Notts County', 'Preston North End', 'Stoke', 'West Bromwich Albion', 'Wolverhampton Wanderers', 'Sunderland', 'Darwen', 'Newton Heath', 'Nottingham Forest', 'The Wednesday', 'Ardwick', 'Bootle', 'Burslem Port Vale', 'Burton Swifts', 'Crewe Alexandra', 'Grimsby Town', 'Lincoln City', 'Northwich Victoria', 'Sheffield United', 'Small Heath', 'Walsall Town Swifts', 'Liverpool', 'Middlesbrough Ironopolis', 'Newcastle United', 'Rotherham Town', 'Walsall', 'Woolwich Arsenal', 'Burton Wanderers', 'Bury', 'Leicester Fosse', 'Manchester City', 'Loughborough', 'Blackpool', 'Gainsborough Trinity', 'Luton Town', 'Barnsley', 'Glossop North End', 'New Brighton Tower', 'Glossop', 'Chesterfield Town', 'Middlesbrough', 'Stockport County', 'Bristol City', 'Burton United', 'Doncaster Rovers', 'Manchester United', 'Bradford City', 'Birmingham', 'Chelsea', 'Clapton Orient', 'Hull City', 'Leeds City',

In [71]:
for t in vertices:
    home = df.loc[df['home_team_name'] == t, "home_points"].sum()
    away = df.loc[df['away_team_name'] == t, "away_points"].sum()
    points = home + away
    print(t, points)

Accrington 138.0
Aston Villa 7075.0
Blackburn Rovers 6985.0
Bolton Wanderers 6874.0
Burnley 6991.0
Derby County 7005.0
Everton 7069.0
Notts County 6687.0
Preston North End 7238.0
Stoke 1182.0
West Bromwich Albion 6907.0
Wolverhampton Wanderers 7265.0
Sunderland 7005.0
Darwen 252.0
Newton Heath 466.0
Nottingham Forest 6773.0
The Wednesday 1785.0
Ardwick 56.0
Bootle 27.0
Burslem Port Vale 507.0
Burton Swifts 305.0
Crewe Alexandra 5417.0
Grimsby Town 6290.0
Lincoln City 6140.0
Northwich Victoria 41.0
Sheffield United 7176.0
Small Heath 687.0
Walsall Town Swifts 18.0
Liverpool 7910.0
Middlesbrough Ironopolis 28.0
Newcastle United 6846.0
Rotherham Town 80.0
Walsall 5783.0
Woolwich Arsenal 1087.0
Burton Wanderers 139.0
Bury 6578.0
Leicester Fosse 1012.0
Manchester City 7191.0
Loughborough 122.0
Blackpool 6486.0
Gainsborough Trinity 643.0
Luton Town 5615.0
Barnsley 6446.0
Glossop North End 66.0
New Brighton Tower 168.0
Glossop 661.0
Chesterfield Town 423.0
Middlesbrough 6397.0
Stockport Count

In [73]:
# https://www.kaggle.com/datasets/kishan305/la-liga-results-19952020?rvi=1

la_liga = pd.read_csv('../data/LaLigaCompleteDataset/LaLiga_Matches.csv', sep=',')
la_liga.head()

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR
0,1995-96,02-09-1995,La Coruna,Valencia,3,0,H,2.0,0.0,H
1,1995-96,02-09-1995,Sp Gijon,Albacete,3,0,H,3.0,0.0,H
2,1995-96,03-09-1995,Ath Bilbao,Santander,4,0,H,2.0,0.0,H
3,1995-96,03-09-1995,Ath Madrid,Sociedad,4,1,H,1.0,1.0,D
4,1995-96,03-09-1995,Celta,Compostela,0,1,A,0.0,0.0,D


In [42]:
la_liga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10883 entries, 0 to 10882
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Season    10883 non-null  object 
 1   Date      10883 non-null  object 
 2   HomeTeam  10883 non-null  object 
 3   AwayTeam  10883 non-null  object 
 4   FTHG      10883 non-null  int64  
 5   FTAG      10883 non-null  int64  
 6   FTR       10883 non-null  object 
 7   HTHG      10881 non-null  float64
 8   HTAG      10881 non-null  float64
 9   HTR       10881 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 850.4+ KB


In [74]:
df = la_liga.copy()
#df = df[df['Season'] == 1995-96]
df = df[['Season', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HTHG','HTAG']] 
df.head()

,Season,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG
0,1995-96,La Coruna,Valencia,3,0,2.0,0.0
1,1995-96,Sp Gijon,Albacete,3,0,3.0,0.0
2,1995-96,Ath Bilbao,Santander,4,0,2.0,0.0
3,1995-96,Ath Madrid,Sociedad,4,1,1.0,1.0
4,1995-96,Celta,Compostela,0,1,0.0,0.0


In [44]:
df['home_points'] = 0.0
df['away_points'] = 0.0

In [75]:
df.loc[df['FTHG'] > df['FTAG'], 'home_points'] = 3
df.loc[df['FTHG'] < df['FTAG'], 'away_points'] = 3
df.loc[df['FTHG'] == df['FTAG'], 'home_points'] = 1
df.loc[df['FTHG'] == df['FTAG'], 'away_points'] = 1
df.head()

,Season,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG,home_points,away_points
0,1995-96,La Coruna,Valencia,3,0,2.0,0.0,3.0,NaN
1,1995-96,Sp Gijon,Albacete,3,0,3.0,0.0,3.0,NaN
2,1995-96,Ath Bilbao,Santander,4,0,2.0,0.0,3.0,NaN
3,1995-96,Ath Madrid,Sociedad,4,1,1.0,1.0,3.0,NaN
4,1995-96,Celta,Compostela,0,1,0.0,0.0,NaN,3.0


In [46]:
df['weights'] = 0.0

In [76]:
df.loc[df['HomeTeam'] > df['AwayTeam'], 'weights'] = 3.0
df.loc[df['HomeTeam'] == df['AwayTeam'], 'weights'] = 1.0
df.head()

,Season,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG,home_points,away_points,weights
0,1995-96,La Coruna,Valencia,3,0,2.0,0.0,3.0,NaN,NaN
1,1995-96,Sp Gijon,Albacete,3,0,3.0,0.0,3.0,NaN,3.0
2,1995-96,Ath Bilbao,Santander,4,0,2.0,0.0,3.0,NaN,NaN
3,1995-96,Ath Madrid,Sociedad,4,1,1.0,1.0,3.0,NaN,NaN
4,1995-96,Celta,Compostela,0,1,0.0,0.0,NaN,3.0,NaN


In [77]:
vertices = list(df['HomeTeam'].unique())
print(vertices)
print(len(vertices))

['La Coruna', 'Sp Gijon', 'Ath Bilbao', 'Ath Madrid', 'Celta', 'Espanol', 'Merida', 'Sevilla', 'Valladolid', 'Vallecano', 'Zaragoza', 'Albacete', 'Barcelona', 'Betis', 'Real Madrid', 'Compostela', 'Oviedo', 'Salamanca', 'Santander', 'Sociedad', 'Tenerife', 'Valencia', 'Hercules', 'Logrones', 'Extremadura', 'Mallorca', 'Alaves', 'Villareal', 'Malaga', 'Numancia', 'Las Palmas', 'Osasuna', 'Villarreal', 'Recreativo', 'Murcia', 'Getafe', 'Levante', 'Cadiz', 'Gimnastic', 'Almeria', 'Xerez', 'Granada', 'Elche', 'Eibar', 'Cordoba', 'Leganes', 'Girona', 'Huesca']
48


In [78]:
for t in vertices:
    home = df.loc[df['HomeTeam'] == t, "home_points"].sum()
    away = df.loc[df['AwayTeam'] == t, "away_points"].sum()
    points = home + away
    print(t, points)

La Coruna 1133.0
Sp Gijon 346.0
Ath Bilbao 1493.0
Ath Madrid 1743.0
Celta 1116.0
Espanol 1294.0
Merida 81.0
Sevilla 1484.0
Valladolid 796.0
Vallecano 626.0
Zaragoza 754.0
Albacete 117.0
Barcelona 2308.0
Betis 1248.0
Real Madrid 2273.0
Compostela 156.0
Oviedo 267.0
Salamanca 105.0
Santander 708.0
Sociedad 1364.0
Tenerife 281.0
Valencia 1718.0
Hercules 76.0
Logrones 33.0
Extremadura 83.0
Mallorca 958.0
Alaves 545.0
Villareal 36.0
Malaga 791.0
Numancia 148.0
Las Palmas 199.0
Osasuna 875.0
Villarreal 1288.0
Recreativo 167.0
Murcia 56.0
Getafe 849.0
Levante 583.0
Cadiz 170.0
Gimnastic 28.0
Almeria 286.0
Xerez 34.0
Granada 364.0
Elche 184.0
Eibar 302.0
Cordoba 20.0
Leganes 159.0
Girona 156.0
Huesca 67.0


In [79]:
# https://www.kaggle.com/datasets/lucmouneyres/ligue-1-saison-20202021?rvi=1

ligue1 = pd.read_csv('../data/Ligue1_saison_2020_2021/data_2020_2021.csv', sep=',')
ligue1.head()

,journee,home,away,scores,home_goal,away_goal,ht_home_goal,ht_away_goal
0,1. Journée,Girondins Bordeaux,FC Nantes,0:0 (0:0),0,0,0,0
1,1. Journée,Dijon FCO,Angers SCO,0:1 (0:1),0,1,0,1
2,1. Journée,Lille OSC,Stade Rennes,1:1 (1:0),1,1,1,0
3,1. Journée,AS Monaco,Stade Reims,2:2 (1:2),2,2,1,2
4,1. Journée,FC Lorient,RC Strasbourg,3:1 (0:1),3,1,0,1


In [51]:
ligue1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   journee       380 non-null    object
 1   home          380 non-null    object
 2   away          380 non-null    object
 3   scores        380 non-null    object
 4   home_goal     380 non-null    int64 
 5   away_goal     380 non-null    int64 
 6   ht_home_goal  380 non-null    int64 
 7   ht_away_goal  380 non-null    int64 
dtypes: int64(4), object(4)
memory usage: 23.9+ KB


In [80]:
df = ligue1.copy()
#df = df[df['journee'] == ]
df = df[['journee', 'home', 'away', 'home_goal', 'away_goal', 'ht_home_goal', 'ht_away_goal']] 
df.head()

,journee,home,away,home_goal,away_goal,ht_home_goal,ht_away_goal
0,1. Journée,Girondins Bordeaux,FC Nantes,0,0,0,0
1,1. Journée,Dijon FCO,Angers SCO,0,1,0,1
2,1. Journée,Lille OSC,Stade Rennes,1,1,1,0
3,1. Journée,AS Monaco,Stade Reims,2,2,1,2
4,1. Journée,FC Lorient,RC Strasbourg,3,1,0,1


In [81]:
df['home_points'] = 0.0
df['away_points'] = 0.0

In [82]:
df.loc[df['home_goal'] > df['away_goal'], 'home_points'] = 3
df.loc[df['home_goal'] < df['away_goal'], 'away_points'] = 3
df.loc[df['home_goal'] == df['away_goal'], 'home_points'] = 1
df.loc[df['home_goal'] == df['away_goal'], 'away_points'] = 1
df.head()

,journee,home,away,home_goal,away_goal,ht_home_goal,ht_away_goal,home_points,away_points
0,1. Journée,Girondins Bordeaux,FC Nantes,0,0,0,0,1.0,1.0
1,1. Journée,Dijon FCO,Angers SCO,0,1,0,1,0.0,3.0
2,1. Journée,Lille OSC,Stade Rennes,1,1,1,0,1.0,1.0
3,1. Journée,AS Monaco,Stade Reims,2,2,1,2,1.0,1.0
4,1. Journée,FC Lorient,RC Strasbourg,3,1,0,1,3.0,0.0


In [83]:
df['weights'] = 0.0

In [84]:
df.loc[df['home'] > df['away'], 'weights'] = 3.0
df.loc[df['home'] == df['away'], 'weights'] = 1.0

In [85]:
vertices = list(df['home'].unique())
print(vertices)
print(len(vertices))

['Girondins Bordeaux', 'Dijon FCO', 'Lille OSC', 'AS Monaco', 'FC Lorient', 'Nîmes Olympique', 'OGC Nice', 'Montpellier HSC', 'Paris Saint-Germain', 'Olympique Marseille', 'Olympique Lyon', 'Stade Rennes', 'RC Strasbourg', 'Stade Reims', 'Angers SCO', 'FC Metz', 'FC Nantes', 'AS Saint-Étienne', 'Stade Brest', 'RC Lens']
20


In [86]:
for t in vertices:
    home = df.loc[df['home'] == t, "home_points"].sum()
    away = df.loc[df['away'] == t, "away_points"].sum()
    points = home + away
    print(t, points)

Girondins Bordeaux 45.0
Dijon FCO 21.0
Lille OSC 83.0
AS Monaco 78.0
FC Lorient 42.0
Nîmes Olympique 35.0
OGC Nice 52.0
Montpellier HSC 54.0
Paris Saint-Germain 82.0
Olympique Marseille 60.0
Olympique Lyon 76.0
Stade Rennes 58.0
RC Strasbourg 42.0
Stade Reims 42.0
Angers SCO 44.0
FC Metz 47.0
FC Nantes 40.0
AS Saint-Étienne 46.0
Stade Brest 41.0
RC Lens 57.0


In [87]:
# https://www.kaggle.com/datasets/mozattt/worldcupdata19302018

worldcup = pd.read_csv('../data/worldcup/WorldCupMatches.csv', sep=',')
worldcup.head()

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA


In [88]:
worldcup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  852 non-null    float64
 1   Datetime              852 non-null    object 
 2   Stage                 852 non-null    object 
 3   Stadium               852 non-null    object 
 4   City                  852 non-null    object 
 5   Home Team Name        852 non-null    object 
 6   Home Team Goals       852 non-null    float64
 7   Away Team Goals       852 non-null    float64
 8   Away Team Name        852 non-null    object 
 9   Win conditions        852 non-null    object 
 10  Attendance            850 non-null    float64
 11  Half-time Home Goals  852 non-null    float64
 12  Half-time Away Goals  852 non-null    float64
 13  Referee               852 non-null    object 
 14  Assistant 1           852 non-null    object 
 15  Assistant 2          

In [90]:
#tennis = pd.read_csv('data/tennis/all_matches.csv', sep=',')
#tennis

In [91]:
# https://www.kaggle.com/datasets/wyattowalsh/basketball

basketeball = pd.read_csv('../data/basketball/game.csv', sep=',')
basketeball.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65698 entries, 0 to 65697
Data columns (total 55 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   season_id               65698 non-null  int64  
 1   team_id_home            65698 non-null  int64  
 2   team_abbreviation_home  65698 non-null  object 
 3   team_name_home          65698 non-null  object 
 4   game_id                 65698 non-null  int64  
 5   game_date               65698 non-null  object 
 6   matchup_home            65698 non-null  object 
 7   wl_home                 65696 non-null  object 
 8   min                     65698 non-null  int64  
 9   fgm_home                65685 non-null  float64
 10  fga_home                50251 non-null  float64
 11  fg_pct_home             50208 non-null  float64
 12  fg3m_home               52480 non-null  float64
 13  fg3a_home               47015 non-null  float64
 14  fg3_pct_home            46624 non-null